# 爬蟲練習

## 郭耀仁

## 批踢踢實業坊

- 熱門看板
- 一般看板
- 十八禁看板

## 熱門看板

- 網址：<https://www.ptt.cc/bbs/hotboards.html>
- 先練習將第一欄：版名抓出來

In [1]:
library(rvest)

url <- "https://www.ptt.cc/bbs/hotboards.html"
raw_html <- read_html(url)
board_css <- ".board-name"
boards <- raw_html %>%
    html_nodes(css = board_css) %>%
    html_text()
head(boards)
length(boards)

Loading required package: xml2


[1] "NBA"       "Gossiping" "Baseball"  "C_Chat"    "LoL"       "sex"

[1] 128

## 熱門看板（2）

- 練習將第二欄：人氣抓出來

```r
library(rvest)

url <- "https://www.ptt.cc/bbs/hotboards.html"
raw_html <- read_html(url)
viewer_css <- "___"
viewers <- raw_html %>%
    html_nodes(css = viewer_css) %>%
    html_text()
head(viewers)
length(viewers)
```

## 熱門看板（3）

- 是否覺得跟第一個欄位的程式很像？
- 我們只需要更換 CSS Selector 或 XPath 而已
- 寫在 for 迴圈裡面，然後用一個函數包起來：

In [2]:
library(rvest)

ptt_hot_boards_crawler <- function(){
  url <- "https://www.ptt.cc/bbs/hotboards.html"
  raw_html <- read_html(url)
  
  board_xpath <- "//a[@class='board']/div[@class='board-name']"
  viewers_xpath <- "//a[@class='board']/div[@class='board-nuser']/span"
  category_xpath <- "//a[@class='board']/div[@class='board-class']"
  title_xpath <- "//a[@class='board']/div[@class='board-title']"
  
  hot_boards_columns <- list()
  columns <- c(board_xpath, viewers_xpath, category_xpath, title_xpath)
  
  for (i in 1:length(columns)){
    content <- raw_html %>%
      html_nodes(xpath = columns[i]) %>%
      html_text()
    hot_boards_columns[[i]] <- content
  }
  names(hot_boards_columns) <- c("board", "viewers", "category", "title")
  return(hot_boards_columns)
}

## 熱門看板（4）

- `ptt_hot_boards_cralwer()` 函數不需要輸入，可以直接呼叫
- 輸入是一個元素有命名清單（list）

In [3]:
current_ptt_hot_boards <- ptt_hot_boards_crawler()
head(current_ptt_hot_boards$board)
head(current_ptt_hot_boards$viewers)
head(current_ptt_hot_boards$category)
head(current_ptt_hot_boards$title)

[1] "NBA"       "Gossiping" "Baseball"  "C_Chat"    "LoL"       "sex"

[1] "29451" "11501" "2379"  "2139"  "1696"  "1618"

[1] "籃球" "綜合" "棒球" "聊天" "遊戲" "男女"

[1] "◎[NBA] 6/10 9:00 Final G4"        "◎這裡是溫馨八卦版"               
[3] "◎[棒球] Yoh回札幌 中職兩地開戰"   "◎[希洽] 2016希洽人氣動畫提名中！"
[5] "◎[LoL] 置底不限制他板難民進駐"    "◎[西斯] 端午徵文活動快截止嚕 ~"

## 熱門看板（5）

- 練習將熱門看板的連結抓出來
- 提示：
    - css selector 改用 `.board`
    - 改用 `html_attr("href")` 而非 `html_text()`

## 一般看板

- 以 NBA 為例：<https://www.ptt.cc/bbs/NBA/index.html>
- 要怎麼將一整頁的文章內容都抓出來呢？

## 一般看板（2）

- 先把看得到的抓出來：
    - 推文數
    - 文章標題
    - ID

In [4]:
library(rvest)

url <- "https://www.ptt.cc/bbs/NBA/index.html"
raw_html <- read_html(url)

nrec_xpath <- "//div[@class='nrec']"
title_xpath <- "//div[@class='title']"
id_xpath <- "//div[@class='meta']/div[@class='author']"

article_info <- list()
columns <- c(nrec_xpath, title_xpath, id_xpath)

for (i in 1:length(columns)){
  content <- raw_html %>%
    html_nodes(xpath = columns[i]) %>%
    html_text()
  article_info[[i]] <- content
}

names(article_info) <- c("nrec", "title", "author_id")

## 一般看板（3）

- 接著我們要清理一下文章標題

In [5]:
head(article_info$title)
head(gsub(pattern = "\n\t+\n\t+", article_info$title, replacement = ""))

[1] "\n\t\t\t\n\t\t\t\t[討論] 騎士穩贏了\n\t\t\t\n\t\t\t"                  
[2] "\n\t\t\t\n\t\t\t\t[討論] 76人是不是2019年要起飛了？\n\t\t\t\n\t\t\t"  
[3] "\n\t\t\t\n\t\t\t\t[討論] 這場騎士的精彩鏡頭要怎麼放？\n\t\t\t\n\t\t\t"
[4] "\n\t\t\t\n\t\t\t\t(本文已被刪除) [kimibear]\n\t\t\t\n\t\t\t"          
[5] "\n\t\t\t\n\t\t\t\t[討論] 柯鋁484只會打順風球?\n\t\t\t\n\t\t\t"        
[6] "\n\t\t\t\n\t\t\t\t裁判是不是真的很會\n\t\t\t\n\t\t\t"

[1] "[討論] 騎士穩贏了"                   "[討論] 76人是不是2019年要起飛了？"  
[3] "[討論] 這場騎士的精彩鏡頭要怎麼放？" "(本文已被刪除) [kimibear]"          
[5] "[討論] 柯鋁484只會打順風球?"         "裁判是不是真的很會"

## 一般看板（4）

- 隨便點一篇文章進去看
- 我們要抓文章的哪些內容？
    - 作者
    - 標題
    - 時間
    - 內文
    - IP
    - 推文標記
    - 推文 ID
    - 推文內容
    - 推文時間

In [6]:
library(rvest)

article_detail <- function(url){
    raw_html <- read_html(url)
    
    author_css <- ".article-metaline:nth-child(1) .article-meta-value"
    title_css <- ".article-metaline-right+ .article-metaline .article-meta-value"
    time_css <- ".article-metaline+ .article-metaline .article-meta-value"
    main_content_css <- "#main-content"
    ip_css <- ".hl+ .f2"
    push_css <- ".push-tag"
    push_id_css <- ".push-userid"
    push_content_css <- ".push-content"
    push_time_css <- ".push-ipdatetime"
    
    article_detail_info <- list()
    columns <- c(author_css, title_css, time_css, main_content_css, ip_css, push_css, push_id_css, push_content_css, push_time_css)
    for (i in 1:length(columns)){
        article_content <- raw_html %>%
            html_nodes(css = columns[i]) %>%
            html_text()
        article_detail_info[[i]] <- article_content
    }
    names(article_detail_info) <- c("author", "title", "time", "main_content", "ip", "push", "push_id", "push_content", "push_time")
    return(article_detail_info)
}

article_url <- "https://www.ptt.cc/bbs/NBA/M.1496931569.A.682.html"
nba_article_details <- article_detail(article_url)

## 一般看板（5）

- 清理內文
- 清理IP
- 清理推文
- 清理推文 ID
- 清理推文內容
- 清理推文時間

## 一般看板（6）

- 清理內文

In [7]:
nba_article_details$main_content <- nba_article_details$main_content %>%
    gsub(pattern = "\n", ., replacement = "") %>% # 清理斷行符號
    gsub(pattern = "--.+", ., replacement = "") %>% # 去尾
    gsub(pattern = "作者.+:[0-9]{2}\\s[0-9]{4}", ., replacement = "") # 去頭

## 一般看板（7）

- 清理IP

In [8]:
ip_start <- regexpr(pattern = "[0-9]+", nba_article_details$ip)
ip_end <- nchar(nba_article_details$ip)
nba_article_details$ip <- substr(nba_article_details$ip, start = ip_start, stop = ip_end)
nba_article_details$ip <- gsub(pattern = "\n", nba_article_details$ip, replacement = "") # 清理斷行符號

## 一般看板（8）

- 清理推文
- 清理推文 ID
- 清理推文內容
- 清理推文時間

In [9]:
nba_article_details$push <- gsub(pattern = "\\s", nba_article_details$push, replacement = "")
nba_article_details$push_id <- gsub(pattern = "\\s", nba_article_details$push_id, replacement = "")
nba_article_details$push_content <- nba_article_details$push_content %>%
    gsub(pattern = "\\s", ., replacement = "") %>%
    gsub(pattern = ":", ., replacement = "")
nba_article_details$push_time <- nba_article_details$push_time %>%
    gsub(pattern = "^\\s", ., replacement = "") %>%
    gsub(pattern = "\n", ., replacement = "")

## 一般看板（8）

- 最後我們把清理資料的程式加入原本的 `article_detail()` 函數中

In [10]:
library(rvest)

article_detail <- function(url){
    raw_html <- read_html(url)
    
    author_css <- ".article-metaline:nth-child(1) .article-meta-value"
    title_css <- ".article-metaline-right+ .article-metaline .article-meta-value"
    time_css <- ".article-metaline+ .article-metaline .article-meta-value"
    main_content_css <- "#main-content"
    ip_css <- ".hl+ .f2"
    push_css <- ".push-tag"
    push_id_css <- ".push-userid"
    push_content_css <- ".push-content"
    push_time_css <- ".push-ipdatetime"
    
    article_detail_info <- list()
    columns <- c(author_css, title_css, time_css, main_content_css, ip_css, push_css, push_id_css, push_content_css, push_time_css)
    for (i in 1:length(columns)){
        article_content <- raw_html %>%
            html_nodes(css = columns[i]) %>%
            html_text()
        article_detail_info[[i]] <- article_content
    }
    
    names(article_detail_info) <- c("author", "title", "time", "main_content", "ip", "push", "push_id", "push_content", "push_time")
    
    article_detail_info$main_content <- article_detail_info$main_content %>%
        gsub(pattern = "\n", ., replacement = "") %>% # 清理斷行符號
        gsub(pattern = "--.+", ., replacement = "") %>% # 去尾
        gsub(pattern = "作者.+:[0-9]{2}\\s[0-9]{4}", ., replacement = "") # 去頭
    ip_start <- regexpr(pattern = "[0-9]+", article_detail_info$ip)
    ip_end <- nchar(article_detail_info$ip)
    article_detail_info$ip <- substr(article_detail_info$ip, start = ip_start, stop = ip_end)
    article_detail_info$ip <- gsub(pattern = "\n", article_detail_info$ip, replacement = "") # 清理斷行符號
    article_detail_info$push <- gsub(pattern = "\\s", article_detail_info$push, replacement = "")
    article_detail_info$push_id <- gsub(pattern = "\\s", article_detail_info$push_id, replacement = "")
    article_detail_info$push_content <- article_detail_info$push_content %>%
        gsub(pattern = "\\s", ., replacement = "") %>%
        gsub(pattern = ":", ., replacement = "")
    article_detail_info$push_time <- article_detail_info$push_time %>%
        gsub(pattern = "^\\s", ., replacement = "") %>%
        gsub(pattern = "\n", ., replacement = "")
    
    return(article_detail_info)
}

## 一般看板（9）

- 接著我們回到最新一頁將所有文章連結抓出來

In [11]:
library(rvest)

url <- "https://www.ptt.cc/bbs/NBA/index.html"
raw_html <- read_html(url)

article_link_xpath <- "//div[@class='title']/a"
article_links <- raw_html %>%
    html_nodes(xpath = article_link_xpath) %>%
    html_attr("href")

In [12]:
head(article_links) # relative url
article_links <- paste("https://www.ptt.cc", article_links, sep = "")
head(article_links) # absolute url

[1] "/bbs/NBA/M.1497066810.A.551.html" "/bbs/NBA/M.1497066958.A.F6A.html"
[3] "/bbs/NBA/M.1497066987.A.31C.html" "/bbs/NBA/M.1497067000.A.8C5.html"
[5] "/bbs/NBA/M.1497067017.A.763.html" "/bbs/NBA/M.1497067034.A.AC1.html"

[1] "https://www.ptt.cc/bbs/NBA/M.1497066810.A.551.html"
[2] "https://www.ptt.cc/bbs/NBA/M.1497066958.A.F6A.html"
[3] "https://www.ptt.cc/bbs/NBA/M.1497066987.A.31C.html"
[4] "https://www.ptt.cc/bbs/NBA/M.1497067000.A.8C5.html"
[5] "https://www.ptt.cc/bbs/NBA/M.1497067017.A.763.html"
[6] "https://www.ptt.cc/bbs/NBA/M.1497067034.A.AC1.html"

## 一般看板（10）

- 一篇文章的詳細內容存在一個清單中
- 外面再用一個清單儲存這些清單（Nested lists）

In [13]:
article_lists <- list()
for (i in 1:length(article_links)){
    article_lists[[i]] <- article_detail(article_links[i])
}

## 十八禁看板

- 十八禁看板會自動跳轉一個確認是否滿十八歲的頁面
- 我們以八卦版（Gossiping）為例：<https://www.ptt.cc/bbs/Gossiping/index.html>

![](img/ch0801.png)

## 十八禁看板（2）

- 瀏覽器的 session 會將使用者點選**我同意**的資訊存在 cookie 中
- 下載一個 Chrome 外掛來觀察：[EditThisCookie](https://chrome.google.com/webstore/search/editthiscookie?hl=zh-TW)

## 十八禁看板（3）

- 點選之後就如同進入一般看板
- 打開 EditThisCookie 外掛可以看到

![](img/ch0802.png)

## 十八禁看板（4）

- 接著我們按上一頁，再連回八卦版
- 這時就沒有跳轉一個確認是否滿十八歲的頁面
- 這是因為 cookie（over18 == 1）已經儲存在 session 中的關係

## 十八禁看板（5）

- 利用 EditThisCookie 將它刪除
- 接著我們按上一頁，再連回八卦版
- 這時就跟我們先前一樣會跳轉一個確認是否滿十八歲的頁面

## 十八禁看板（6）

- 回到 R 語言，如果我們將十八禁看板的網址直接傳入
- 將會因為跳轉頁面的關係無法到達看板的最新文章頁面

In [14]:
url <- "https://www.ptt.cc/bbs/Gossiping/index.html"
ptt_gossiping <- readLines(url)

## 十八禁看板（7）

- 這時我們要使用 `httr` 套件所提供的 `GET()` 函數幫忙設定 cookie

```
install.packages("httr")
```

In [15]:
# install.packages("httr")
library(httr)

url <- "https://www.ptt.cc/bbs/Gossiping/index.html"
ptt_gossiping <- GET(url, set_cookies(over18 = 1))

## 十八禁看板（8）

- 在一般看板的程式中加入 `httr` 的 `GET()` 函數即可

In [16]:
library(rvest)
library(httr)

ptt_index_crawler <- function(url){
    ptt_index_html_doc <- url %>%
        GET(set_cookies(over18 = 1)) %>%
        read_html()
    nrec_xpath <- "//div[@class='nrec']"
    title_xpath <- "//div[@class='title']"
    id_xpath <- "//div[@class='meta']/div[@class='author']"
    article_info <- list()
    columns <- c(nrec_xpath, title_xpath, id_xpath)
    for (i in 1:length(columns)){
        content <- ptt_index_html_doc %>%
            html_nodes(xpath = columns[i]) %>%
            html_text()
        article_info[[i]] <- content
    }
    names(article_info) <- c("nrec", "title", "author_id")
    article_info$title <- gsub(pattern = "\n\t+\n\t+", article_info$title, replacement = "")
    return(article_info)
}

ptt_gossiping_index <- ptt_index_crawler("https://www.ptt.cc/bbs/Gossiping/index.html")
ptt_nba_index <- ptt_index_crawler("https://www.ptt.cc/bbs/NBA/index.html")